In [4]:
# Step 1 Setup Openpose
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if True:
    # # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
    # # install new CMake becaue of CUDA10
    # !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
    # !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
    # clone openpose
    !git clone - q - -depth 1 $git_repo_url
    !sed - i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
    # install system dependencies
    !apt-get - qq install - y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
    # install python dependencies
    !pip install - q youtube-dl
    # build openpose
    !cd openpose & & rm - rf build | | true & & mkdir build & & cd build & & cmake .. & & make - j`nproc`


fatal: destination path 'openpose' already exists and is not an empty directory.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
CMake Warning (dev) at CMakeLists.txt:19 (project):
  cmake_minimum_required() should be called prior to this top-level project()
  call.  Please see the cmake-commands(7) manual for usage documentation of
  both commands.
This warning is for project developers.  Use -Wno-dev to suppress it.

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX comp

In [ ]:
# S1 run openpose


In [1]:
# Step 4 Setup ClothMask
from carvekit.ml.files.models_loc import download_all
!pip install carvekit_colab
download_all()


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


/home/dwang/miniconda3/envs/dci-vton/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
"""
Input cloth image.
Output cloth mask. 
"""
# title Upload images from your computer
# markdown Description of parameters
# markdown - `SHOW_FULLSIZE`  - Shows image in full size (may take a long time to load)
# markdown - `PREPROCESSING_METHOD`  - Preprocessing method
# markdown - `SEGMENTATION_NETWORK`  - Segmentation network. Use `u2net` for hairs-like objects and `tracer_b7` for objects
# markdown - `POSTPROCESSING_METHOD`  - Postprocessing method
# markdown - `SEGMENTATION_MASK_SIZE` - Segmentation mask size. Use 640 for Tracer B7 and 320 for U2Net
# markdown - `TRIMAP_DILATION`  - The size of the offset radius from the object mask in pixels when forming an unknown area
# markdown - `TRIMAP_EROSION`  - The number of iterations of erosion that the object's mask will be subjected to before forming an unknown area

import os
import numpy as np
from PIL import Image, ImageOps
from carvekit.web.schemas.config import MLConfig
from carvekit.web.utils.init_utils import init_interface

SHOW_FULLSIZE = False  # param {type:"boolean"}
PREPROCESSING_METHOD = "none"  # param ["stub", "none"]
# param ["u2net", "deeplabv3", "basnet", "tracer_b7"]
SEGMENTATION_NETWORK = "tracer_b7"
POSTPROCESSING_METHOD = "fba"  # param ["fba", "none"]
# param ["640", "320"] {type:"raw", allow-input: true}
SEGMENTATION_MASK_SIZE = 640
TRIMAP_DILATION = 30  # param {type:"integer"}
TRIMAP_EROSION = 5  # param {type:"integer"}
DEVICE = 'cuda'  # 'cuda'

background = 130

config = MLConfig(segmentation_network=SEGMENTATION_NETWORK,
                  preprocessing_method=PREPROCESSING_METHOD,
                  postprocessing_method=POSTPROCESSING_METHOD,
                  seg_mask_size=SEGMENTATION_MASK_SIZE,
                  trimap_dilation=TRIMAP_DILATION,
                  trimap_erosion=TRIMAP_EROSION,
                  device=DEVICE)

interface = init_interface(config)

imgs = []
root = '/opt/disk1/dwang/sci/HR-VITON/cloth'
for name in os.listdir(root):
    imgs.append(root + '/' + name)

images = interface(imgs)

for i, im in enumerate(images):
    img = np.array(im)
    img = img[..., :3]  # no transparency
    idx = (img[..., 0] == background) & (img[..., 1] == background) & (
        img[..., 2] == background)  # background 0 or 130, just try it
    img = np.ones(idx.shape)*255
    img[idx] = 0
    im = Image.fromarray(np.uint8(img), 'L')
    im.save(
        f'/opt/disk1/dwang/sci/HR-VITON/cloth_mask/mask_{imgs[i].split("/")[-1].split(".")[0]}.jpg')
    # import pdb
    # pdb.set_trace()


In [10]:
# S5 Parse Agnostic
!pip install Pillow tqdm


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
import json
from os import path as osp
import os
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm


def visulize_agnostic(agnostic_img, image_path):
    np_im = np.array(agnostic_img)
    np_im[np_im == 2] = 151
    np_im[np_im == 9] = 178
    np_im[np_im == 13] = 191
    np_im[np_im == 14] = 221
    np_im[np_im == 15] = 246
    image_vis = Image.fromarray(np_im)
    image_vis.save(image_path)


def get_im_parse_agnostic(im_parse, pose_data, w=768, h=1024):
    label_array = np.array(im_parse)
    parse_upper = ((label_array == 5).astype(np.float32) +
                   (label_array == 6).astype(np.float32) +
                   (label_array == 7).astype(np.float32))
    parse_neck = (label_array == 10).astype(np.float32)

    r = 10
    agnostic_img = im_parse.copy()

    # mask arms
    for parse_id, pose_ids in [(14, [2, 5, 6, 7]), (15, [5, 2, 3, 4])]:
        mask_arm = Image.new('L', (w, h), 'black')
        mask_arm_draw = ImageDraw.Draw(mask_arm)
        i_prev = pose_ids[0]
        for i in pose_ids[1:]:
            if (pose_data[i_prev, 0] == 0.0 and pose_data[i_prev, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
                continue
            mask_arm_draw.line([tuple(pose_data[j])
                               for j in [i_prev, i]], 'white', width=r*10)
            pointx, pointy = pose_data[i]
            radius = r*4 if i == pose_ids[-1] else r*15
            mask_arm_draw.ellipse(
                (pointx-radius, pointy-radius, pointx+radius, pointy+radius), 'white', 'white')
            i_prev = i
        parse_arm = (np.array(mask_arm) / 255) * \
            (label_array == parse_id).astype(np.float32)
        agnostic_img.paste(0, None, Image.fromarray(
            np.uint8(parse_arm * 255), 'L'))

    # mask torso & neck
    agnostic_img.paste(0, None, Image.fromarray(
        np.uint8(parse_upper * 255), 'L'))
    agnostic_img.paste(0, None, Image.fromarray(
        np.uint8(parse_neck * 255), 'L'))

    return agnostic_img


if __name__ == "__main__":
    data_path = '/opt/disk1/dwang/sci/datasets'
    output_path = '/opt/disk1/dwang/sci/datasets/image-parse-agnostic-v3.2'

    os.makedirs(output_path, exist_ok=True)

    for im_name in tqdm(os.listdir(osp.join(data_path, 'image'))):
        # load pose image
        pose_name = im_name.replace('.jpg', '_keypoints.json')
        try:
            with open(osp.join(data_path, 'openpose_json', pose_name), 'r') as f:
                pose_label = json.load(f)
                pose_data = pose_label['people'][0]['pose_keypoints_2d']
                pose_data = np.array(pose_data)
                pose_data = pose_data.reshape((-1, 3))[:, :2]
        except IndexError:
            print("index_error for", pose_name)
            continue

        # load parsing image
        parse_name = im_name.replace('.jpg', '.png')
        im_parse = Image.open(
            osp.join(data_path, 'image-parse-v3', parse_name))

        agnostic_img = get_im_parse_agnostic(im_parse, pose_data)

        agnostic_img.save(osp.join(output_path, parse_name))
        vis_name = osp.join(output_path, "vis_"+parse_name)
        visulize_agnostic(agnostic_img, vis_name)
        # Check parse result

        print(np.unique(np.array(im_parse)))
        print(np.unique(np.array(agnostic_img)))


 80%|████████  | 4/5 [00:00<00:00, 16.27it/s]

[ 0  2  5  9 10 13 14]
[ 0  2  9 13 14]
[ 0  2  5  9 10 12 13 14 15 16 17]
[ 0  2  9 12 13 14 15 16 17]
[ 0  2  5  9 10 13 14 15]
[ 0  2  9 13 14 15]
[ 0  2  5  9 10 13 14 15]
[ 0  2  9 13 14 15]


100%|██████████| 5/5 [00:00<00:00, 16.20it/s]

[ 0  2  5  9 10 12 13 14 15 16 17]
[ 0  2  9 12 13 14 15 16 17]


In [5]:
# 6 Human Agnostic
import json
from os import path as osp
import os
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm


def get_img_agnostic(img, parse, pose_data):
    parse_array = np.array(parse)
    parse_head = ((parse_array == 4).astype(np.float32) +
                  (parse_array == 13).astype(np.float32))
    parse_lower = ((parse_array == 9).astype(np.float32) +
                   (parse_array == 12).astype(np.float32) +
                   (parse_array == 16).astype(np.float32) +
                   (parse_array == 17).astype(np.float32) +
                   (parse_array == 18).astype(np.float32) +
                   (parse_array == 19).astype(np.float32))

    agnostic = img.copy()
    agnostic_draw = ImageDraw.Draw(agnostic)

    length_a = np.linalg.norm(pose_data[5] - pose_data[2])
    length_b = np.linalg.norm(pose_data[12] - pose_data[9])
    point = (pose_data[9] + pose_data[12]) / 2
    pose_data[9] = point + (pose_data[9] - point) / length_b * length_a
    pose_data[12] = point + (pose_data[12] - point) / length_b * length_a
    r = int(length_a / 16) + 1

    # mask arms
    agnostic_draw.line([tuple(pose_data[i])
                       for i in [2, 5]], 'gray', width=r*10)
    for i in [2, 5]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse(
            (pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')
    for i in [3, 4, 6, 7]:
        if (pose_data[i - 1, 0] == 0.0 and pose_data[i - 1, 1] == 0.0) or (pose_data[i, 0] == 0.0 and pose_data[i, 1] == 0.0):
            continue
        agnostic_draw.line([tuple(pose_data[j])
                           for j in [i - 1, i]], 'gray', width=r*10)
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse(
            (pointx-r*5, pointy-r*5, pointx+r*5, pointy+r*5), 'gray', 'gray')

    # mask torso
    for i in [9, 12]:
        pointx, pointy = pose_data[i]
        agnostic_draw.ellipse(
            (pointx-r*3, pointy-r*6, pointx+r*3, pointy+r*6), 'gray', 'gray')
    agnostic_draw.line([tuple(pose_data[i])
                       for i in [2, 9]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i])
                       for i in [5, 12]], 'gray', width=r*6)
    agnostic_draw.line([tuple(pose_data[i])
                       for i in [9, 12]], 'gray', width=r*12)
    agnostic_draw.polygon([tuple(pose_data[i])
                          for i in [2, 5, 12, 9]], 'gray', 'gray')

    # mask neck
    pointx, pointy = pose_data[1]
    agnostic_draw.rectangle(
        (pointx-r*7, pointy-r*7, pointx+r*7, pointy+r*7), 'gray', 'gray')
    agnostic.paste(img, None, Image.fromarray(np.uint8(parse_head * 255), 'L'))
    agnostic.paste(img, None, Image.fromarray(
        np.uint8(parse_lower * 255), 'L'))

    return agnostic


if __name__ == "__main__":
    data_path = '/opt/disk1/dwang/sci/datasets/'
    output_path = '/opt/disk1/dwang/sci/datasets/agnostic-v3.2'

    os.makedirs(output_path, exist_ok=True)

    for im_name in tqdm(os.listdir(osp.join(data_path, 'image'))):

        # load pose image
        pose_name = im_name.replace('.jpg', '_keypoints.json')

        try:
            with open(osp.join(data_path, 'openpose_json', pose_name), 'r') as f:
                pose_label = json.load(f)
                pose_data = pose_label['people'][0]['pose_keypoints_2d']
                pose_data = np.array(pose_data)
                pose_data = pose_data.reshape((-1, 3))[:, :2]
        except IndexError:
            print(pose_name)
            continue

        # load parsing image
        im = Image.open(osp.join(data_path, 'image', im_name))
        label_name = im_name.replace('.jpg', '.png')
        im_label = Image.open(
            osp.join(data_path, 'image-parse-v3', label_name))

        agnostic = get_img_agnostic(im, im_label, pose_data)

        agnostic.save(osp.join(output_path, im_name))


100%|██████████| 5/5 [00:00<00:00, 27.37it/s]
